In [1]:
import numpy as np
import chainer
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import link, Chain, ChainList
import chainer.function as F
import chainer.link as L
from chainer.training import extensions